# tutorial3.ipynb

A network of excitatory and inhibitory units with plastic synapses.

The previous tutorials have already provided all the tools required to create more complex simulations. In this tutorial a network with the following properties will be created:
* It will have 3 populations of units, called `exc`, `inh`, and `inp`. Those populations will have excitatory, inhibitory, and source units respectively.
* Each population will be arranged in a NxN grid where each unit is 1 length unit away from its neighbours. The grids have periodic boundaries, so units at the very top of the grid are nearby units at the bottom, and units at the far left are close to units on the right.
* Excitatory units send projections to nearby excitatory units (using a 'disk' mask), and to inhibitory units at intermediate distances (using an 'annular' mask).
* Inhibitory units send projections

It will become obvious that writing this simulation involves typing a lot of parameters, and writing code to visualize things. For these reason we create classes that help with these tasks.